In [1]:
import torch
import torch.nn.functional as F

def apply_temperature(logits, temperature):
    # টেম্পারেচার ১.০ হলে কিছুই বদলাবে না
    # ১.০ এর কম হলে লজিকগুলো শার্প হবে (Safe)
    # ১.০ এর বেশি হলে লজিকগুলো ফ্ল্যাট হবে (Creative/Random)
    return logits / temperature

In [8]:
dummy_logits = torch.tensor([[1.0, 2.0, 5.0, 0.5, 0.1, 0.1, 0.1, 4.0, 0.2, 0.3]])
    
print("Original Logits:", dummy_logits)
    
# Test 1: Temperature (High) -> ভ্যালুগুলো কাছাকাছি চলে আসবে
print("\nHigh Temp (2.0):",apply_temperature(dummy_logits.clone(), 0.8))

Original Logits: tensor([[1.0000, 2.0000, 5.0000, 0.5000, 0.1000, 0.1000, 0.1000, 4.0000, 0.2000,
         0.3000]])

High Temp (2.0): tensor([[1.2500, 2.5000, 6.2500, 0.6250, 0.1250, 0.1250, 0.1250, 5.0000, 0.2500,
         0.3750]])


In [8]:
def apply_top_k(logits, k):
    # সেরা k-টি ভ্যালু রেখে বাকি সব -Infinity করে দেওয়া
    if k is None or k == 0:
        return logits
            
    # সেরা k-তম ভ্যালুটা খুঁজে বের করা
    top_val, _ = torch.topk(logits, k)
    min_top_val = top_val[:, -1].unsqueeze(-1) # সবচেয়ে ছোটটি (k-th)
        
    # যেগুলোর ভ্যালু k-তম ভ্যালুর চেয়ে কম, তাদের বাদ দেওয়া (-Inf)
    logits[logits < min_top_val] = float('-inf')
    return logits

In [9]:
print("\nTop-k (k=3):", apply_top_k(dummy_logits.clone(), k=3))


Top-k (k=3): tensor([[-inf, 2., 5., -inf, -inf, -inf, -inf, 4., -inf, -inf]])


In [29]:
x = torch.tensor([[1., 2., 8., 5.]])
top_val, _ = torch.topk(x, k=2)
print(top_val, "\nPosition : ", _)

tensor([[8., 5.]]) 
Position :  tensor([[2, 3]])


In [30]:
min_top_val = top_val[:, -1].unsqueeze(-1)
min_top_val

x[x < min_top_val] = float('-inf')
x

tensor([[-inf, -inf, 8., 5.]])

In [32]:
def apply_top_p(logits, p):
    """
    Nucleus Sampling: Cumulative Probability যোগ করে p (যেমন 0.9) পর্যন্ত নেওয়া।
    """
    if p is None or p >= 1.0:
        return logits

    # ১. লজিটস সর্ট করা (বড় থেকে ছোট)
    sorted_logits, sorted_indices = torch.sort(logits, descending=True, dim=-1)
    
    # ২. সফটম্যাক্স করে প্রোবাবিলিটি বের করা
    sorted_probs = F.softmax(sorted_logits, dim=-1)
    
    # ৩. কিউমুলেটিভ সাম (Cumulative Sum) বের করা
    # [0.5, 0.3, 0.1, ...] -> [0.5, 0.8, 0.9, ...]
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    
    # ৪. মাস্ক তৈরি করা: যারা p এর চেয়ে বেশি, তাদের বাদ দেওয়া
    # আমরা চাই প্রথম যেই ইনডেক্সে যোগফল > p হবে, তার পরেরগুলো বাদ যাবে
    sorted_indices_to_remove = cumulative_probs > p
    
    # ট্রিক: মাস্ক এক ঘর ডানে শিফট করা, যাতে প্রথম ভ্যালুটা (যেটা p ক্রস করল) টিকে থাকে
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0  # প্রথম সেরা শব্দটি সবসময় রাখব
    
    # ৫. অরিজিনাল লজিটস-এ ফেরত গিয়ে বাদ দেওয়া অংশ -Inf করা
    # scatter ব্যবহার করে সর্টেড ইনডেক্স অনুযায়ী অরিজিনাল জায়গায় মাস্ক বসানো
    indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
    logits[indices_to_remove] = float('-inf')
    
    return logits

In [33]:
print("\nTop-p (p=0.8):", apply_top_p(dummy_logits.clone(), p=0.8))


Top-p (p=0.8): tensor([[-inf, -inf, 5., -inf, -inf, -inf, -inf, 4., -inf, -inf]])


In [50]:
x2 = torch.tensor([[1., 2., 8., 5.]])
sorted_logits, sorted_indices = torch.sort(x2, descending=True, dim=-1)

In [51]:
print("\n",sorted_logits, "\n", sorted_indices)


 tensor([[8., 5., 2., 1.]]) 
 tensor([[2, 3, 1, 0]])


In [52]:
sorted_probs = F.softmax(sorted_logits, dim=-1)
sorted_probs

tensor([[9.4951e-01, 4.7273e-02, 2.3536e-03, 8.6584e-04]])

In [58]:
# ৩. কিউমুলেটিভ সাম (Cumulative Sum) বের করা
# [0.5, 0.3, 0.1, ...] -> [0.5, 0.8, 0.9, ...]
cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
cumulative_probs

tensor([[0.9495, 0.9968, 0.9991, 1.0000]])

In [59]:
# ৪. মাস্ক তৈরি করা: যারা p এর চেয়ে বেশি, তাদের বাদ দেওয়া
# আমরা চাই প্রথম যেই ইনডেক্সে যোগফল > p হবে, তার পরেরগুলো বাদ যাবে
sorted_indices_to_remove = cumulative_probs > 0.95

sorted_indices_to_remove

tensor([[False,  True,  True,  True]])

In [60]:
# ট্রিক: মাস্ক এক ঘর ডানে শিফট করা, যাতে প্রথম ভ্যালুটা (যেটা p ক্রস করল) টিকে থাকে
sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
sorted_indices_to_remove[..., 0] = 0  # প্রথম সেরা শব্দটি সবসময় রাখব

sorted_indices_to_remove

tensor([[False, False,  True,  True]])

In [61]:
# ৫. অরিজিনাল লজিটস-এ ফেরত গিয়ে বাদ দেওয়া অংশ -Inf করা
# scatter ব্যবহার করে সর্টেড ইনডেক্স অনুযায়ী অরিজিনাল জায়গায় মাস্ক বসানো
indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)

indices_to_remove


tensor([[ True,  True, False, False]])

In [57]:
x2[indices_to_remove] = float('-inf')

x2

tensor([[-inf, -inf, 8., 5.]])